In [318]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [319]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [320]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_without_reviews

data = json.loads(json_util.dumps(input_data.find()))

In [321]:
data2 = pd.DataFrame(json_normalize(data))

In [322]:
ing = pd.DataFrame(json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore'))


nutritions = pd.DataFrame(json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count']))


In [323]:
#  ------  erstellung und data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samt ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)


In [324]:
#  ------ Erstellung und cleansing des Nutrition Dataframes

# erstelle neue liste auf basis der bereits gefilterten rezepte aus ingredients_filt
id_overlap_mask = nutritions['id'].isin(ingredients_filt.index.get_level_values('id').values)

# erstelle datenframe auf basis der overlapliste
nutritions_filt = nutritions.loc[id_overlap_mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)

# remove multiindex 'amount'
nutrition_db.columns = nutrition_db.columns.droplevel(0)

# entferne alle NA
nutrition_db = nutrition_db.dropna()

# gleiche nochmals die ids der beiden dataframe nutrition und ingredients ab, da der nutrition dataframe noch NA Werte hatt
id_overlap_mask = ingredients_filt.index.get_level_values('id').isin(nutrition_db.index)
ingredients_db = ingredients_filt[id_overlap_mask]

# abgleich ob anzahl der indizes von nutrition und zutaten dataframe gleich sind


In [325]:


ingredients_db.reset_index(inplace=True)

recipe_db = pd.get_dummies(ingredients_db['ingredients_id']).groupby(
    ingredients_db['id']).apply(max)



In [326]:
new_ingredients = ingredients_db.copy()
#new_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index()
gramms_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index().copy()

Q1 = gramms_ingredients.quantile(0.25)
Q3 = gramms_ingredients.quantile(0.75)
IQR = Q3 - Q1

#Filter out all recipes which are outlier by their weight (gramms)
df = gramms_ingredients[((gramms_ingredients >= (Q1 - 1.5 * IQR))& (gramms_ingredients <= (Q3 + 1.5 * IQR))).all(axis=1)].copy()

#filter out recipes by weight which are not in the range 500 - 2373.59 gramms
df_start_at_fivehundret = df[df['ingredients_grams'].between(500, 2373.58225, inclusive=True)].copy()

df_start_at_fivehundret.set_index('id', inplace=True)
id_overlap_mask = nutritions['id'].isin(df_start_at_fivehundret.index.get_level_values('id').values)


# erstelle datenframe auf basis der overlapliste
nutritions_filt_gramm = nutritions.loc[id_overlap_mask]

nutrition_db2 = nutritions_filt_gramm.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

#create new nutrition db based on the above filtering
nutrition_db2.set_index('id', inplace=True)
nutrition_db2.columns = nutrition_db2.columns.droplevel(0)



In [327]:
cluster1_ids = [  8493,   8500,   8519,   8522,   8532,   8542,   8543,   8547,
         8564,   8577,   8597,   8599,   8600,   8620,   8633,   8635,
         8641,   8653,   8654,   8656,   8662,   8669,   8673,   8685,
         8689,   8694,   8697,   8701,   8702,   8706,   8710,   8712,
         8714,   8717,   8719,   8721,   8729,   8731,   8739,   8750,
         8751,   8756,   8757,   8763,   8765,   8772,   8776,   8783,
         8802,   8805,   8807,   8808,   8820,   8826,   8829,   8836,
         8841,   8853,   8854,   8855,   8858,   8862,   8873,   8876,
         8877,   8887,   8889,   8895,   8898,   8901,   8907,   8918,
         8925,   8928,   8937,   8944,   8945,   8961,   8962,   8964,
         8965,   8969,   8974,   8990,   8994,   9000,   9003,   9005,
         9006,   9010,   9024,   9026,   9032,   9039,   9043,   9059,
         9143,   9340,   9411,  11678,  11679,  11683,  11726,  11735,
        11736,  11739,  11743,  11746,  11751,  11760,  11763,  11767,
        11805,  11827,  11832,  11880,  11905,  11909,  11911,  11914,
        11921,  11927,  11937,  11941,  11961,  11962,  11968,  11970,
        11983,  11989,  11990,  11999,  12012,  12016,  12039,  12051,
        12059,  12066,  12281,  12475,  12508,  12522,  12668,  12704,
        12731,  12734,  12736,  12772,  12807,  12823,  12825,  12830,
        12851,  12854,  12859,  12865,  13239,  13420,  13638,  13664,
        13856,  13890,  13934,  13938,  13941,  14114,  14140,  14499,
        14518,  14543,  14586,  14587,  14588,  14593,  14603,  14604,
        14614,  14618,  14626,  14628,  14629,  14636,  14642,  14656,
        14670,  14677,  14684,  14687,  14690,  14693,  14705,  14717,
        14724,  14735,  14739,  14762,  14764,  14770,  14772,  14785,
        14965,  14979,  15000,  15123,  15127,  15132,  15134,  15167,
        15181,  15194,  15196,  15338,  15364,  15370,  15375,  15378,
        15509,  15523,  15604,  15640,  15698,  15919,  16092,  16189,
        16196,  16220,  16224,  16237,  16256,  16258,  16278,  16304,
        16346,  16366,  16377,  16378,  16381,  16388,  16399,  16433,
        16470,  16473,  16553,  16561,  16563,  16566,  16576,  16611,
        16650,  16669,  16681,  16700,  16703,  16731,  16732,  16781,
        16786,  16794,  16801,  16824,  16861,  16884,  16974,  16976,
        16978,  16998,  17022,  17028,  17040,  17053,  17072,  17118,
        17169,  17170,  17178,  17180,  17198,  17219,  17253,  17272,
        17315,  17323,  17426,  17437,  17483,  17525,  17532,  17546,
        17646,  17648,  17672,  17717,  17737,  17753,  17837,  17863,
        17952,  17996,  17997,  18004,  18009,  18044,  18078,  18079,
        18235,  18263,  18287,  18349,  18363,  18439,  18440,  18465,
        18466,  18795,  18803,  18805,  18806,  18815,  18821,  18840,
        18841,  18882,  19126,  19128,  19301,  19304,  19312,  19324,
        19347,  19364,  19368,  19373,  19399,  19415,  19417,  19422,
        19423,  19427,  19497,  19521,  19851,  19889,  19891,  19905,
        19912,  20004,  20059,  20061,  20100,  20107,  20229,  20458,
        20460,  20502,  20606,  20618,  20675,  20680,  20684,  20687,
        20744,  20778,  20817,  20852,  20921,  21122,  21150,  21253,
        21303,  21360,  21362,  21441,  21497,  21503,  21507,  21518,
        21541,  21575,  21582,  21605,  21669,  21705,  21728,  21732,
        21777,  21862,  22179,  22204,  22261,  22273,  22292,  22329,
        22364,  22417,  22448,  22529,  22634,  22644,  22671,  22679,
        22725,  22805,  22884,  22921,  22973,  22991,  23018,  23041,
        23048,  23058,  23067,  23070,  23101,  23105,  23147,  23157,
        23197,  23260,  23340,  23352,  23434,  23550,  23563,  23582,
        23628,  23691,  23696,  23705,  23725,  23811,  23833,  23840,
        23849,  23850,  23924,  23980,  23984,  24009,  24075,  24080,
        24160,  24202,  24263,  24321,  24322,  24512,  24520,  24558,
        24674,  24675,  24679,  24681,  24682,  24683,  24684,  24778,
        24783,  24786,  24788,  24793,  24807,  24848,  24852,  25190,
        25204,  25206,  25303,  25405,  25409,  25415,  25416,  25423,
        25473,  25490,  25710,  25723,  25726,  25860,  25888,  25927,
        26260,  26374,  26599,  26607,  26609,  26611,  26614,  26615,
        26619,  26640,  26641,  26642,  26656,  26661,  26667,  26672,
        27074,  27838,  27987,  28078,  28347,  29529,  30007,  30575,
        30679,  30693,  30826,  31045,  31064,  31789,  31796,  32003,
        32209,  34159,  34357,  34361,  34365,  34700,  34825,  34892,
        35469,  35855,  36079,  36116,  36763,  36823,  38004,  38276,
        38389,  39501,  39533,  40156,  40292,  40422,  40545,  40573,
        41431,  41444,  41550,  41690,  42814,  42824,  42890,  42919,
        42967,  43272,  43711,  43742,  43834,  44525,  44839,  45597,
        45827,  45933,  46604,  46813,  46922,  47374,  47407,  47525,
        47688,  48477,  48788,  48931,  49404,  49497,  49735,  49925,
        50384,  50436,  50919,  51156,  51773,  52310,  52464,  52501,
        52552,  52755,  52934,  53194,  53367,  53613,  54067,  54143,
        54215,  54235,  54496,  54507,  54994,  55593,  55860,  56306,
        56311,  57372,  58348,  58793,  58831,  58960,  59303,  59507,
        59661,  59988,  60022,  60088,  60704,  60923,  61341,  61645,
        62256,  62459,  64302,  64311,  64337,  64539,  64979,  65016,
        66623,  68380,  68429,  68479,  68578,  68613,  68806,  69005,
        69119,  70012,  70096,  70490,  70513,  70562,  70638,  70855,
        70985,  71291,  71380,  71418,  71484,  71555,  72012,  72068,
        72191,  72277,  72392,  72421,  72470,  72525,  72595,  72715,
        72797,  72845,  73604,  73910,  74053,  74142,  74144,  74188,
        74225,  75453,  75672,  75700,  76089,  76129,  76301,  76487,
        76586,  77013,  77799,  78493,  79385,  79601,  80200,  80398,
        80984,  81222,  81525,  83440,  83646,  83701,  83746,  83823,
        84043,  84044,  84136,  84327,  84357,  84482,  84717,  84720,
        84745,  85031,  85034,  85145,  85765,  86047,  86069,  86168,
        86371,  86775,  86836,  87025,  87133,  87149,  87173,  87524,
        87860,  88281,  88768,  89721,  90223,  90277,  90481,  90497,
        90530,  91577,  92525,  92528,  92634,  92992,  93223,  93770,
        94894,  95091, 100195, 102235, 102393, 102765, 103144, 103503,
       104850, 105016, 105058, 105434, 106528, 109642, 109925, 111823,
       112206, 117539, 125646, 125658, 125713, 127500, 131713, 132576,
       133034, 133042, 134291, 135645, 139065, 139948, 139972, 140829,
       142220, 142773, 143078, 143473, 146092, 146125, 147305, 147633,
       147743, 148794, 149474, 149686, 149738, 150156, 150357, 151027,
       151153, 151340, 152184, 152669, 155074, 155190, 155498, 157173,
       158587, 159456, 161469, 161869, 162690, 163894, 164920, 164957,
       165527, 169293, 169613, 170061, 171435, 172100, 172772, 176132,
       177115, 177708, 177777, 177809, 178033, 178809, 180596, 180905,
       183650, 183947, 184585, 185493, 185816, 186983, 188706, 196428,
       201934, 202265, 203047, 203752, 204176, 204353, 204778, 206120,
       206688, 211793, 212451, 212479, 212511, 213206, 213210, 214045,
       214502, 214736, 214820, 214823, 214945, 215983, 216078, 216111,
       216355, 216470, 216899, 216908, 216914, 216928, 217123, 217151,
       217182, 217272, 218043, 218658, 218863, 218950, 218982, 219046,
       219072, 219166, 219758, 219802, 219919, 220003, 220059, 220160,
       220346, 220376, 220490, 220515, 220639, 220716, 220851, 220865,
       220872, 221001, 221080, 221227, 221256, 221509, 222037, 222178,
       222181, 222238, 222404, 222405, 222509, 222583, 222610, 222637,
       222973, 223218, 227993, 228045, 228134, 228402, 228431, 228450,
       228680, 228858, 229057, 229090, 229099, 229123, 229277, 229324,
       229361, 229486, 229566, 229616, 229964, 229994, 230249, 230283,
       230301, 230302, 230567, 230586, 230832, 230957, 231064, 231094,
       231233, 231417, 231533, 231535, 231565, 231808, 231880, 232080,
       232097, 232102, 232227, 232355, 232458, 232703, 232838, 232908,
       233065, 233300, 233391, 233480, 233999, 234803, 235089, 235250,
       235380, 235432, 235455, 235588, 235710, 235736, 235924, 235997,
       236001, 236013, 236128, 236131, 236359, 236433, 236502, 236703,
       236709, 236805, 236867, 236919, 236942, 237320, 237406, 237499,
       238100, 238691, 239198, 239230, 239533, 239601, 239909, 240391,
       240502, 240568, 240619, 240773, 241287, 244950, 245764, 245946,
       254417, 254812, 260193, 263813]

cluster1 = nutrition_db2[nutrition_db2.index.isin(cluster1_ids)]

In [328]:
data2.set_index('id', inplace=True)

In [329]:
# random
subset_random_top10 = nutrition_db2.sample(n=10).copy()

In [330]:
subset_random_top10.index.to_numpy()

array([ 70491,  21815,  58165,  17646,  16767,  90451, 229277,  15919,
       148970,  88745])

In [331]:
clusttest = data2[data2.index.isin(cluster1_ids)]

In [332]:
subset_fat = cluster1[(cluster1['Fat'] >= 35) & (cluster1['Fat'] <= 40)].sample(n=40).copy()
subset_fat_id = subset_fat.index.to_numpy()

In [423]:
subset_fat_id = [38004, 237320,  16563, 142220, 106528,  72277, 228431, 228134,

       149738,  17253]

In [424]:
#subset_normal = nutrition_db2.sample(n=40).copy()
#subset_normal_id = subset_normal.index.to_numpy()

In [425]:
#subset_normal.index.to_numpy()

In [426]:
subset_fat.index.to_numpy()

array([ 84044, 236703,   8772, 222509,  16794, 111823,  23849,  72191,
       229277,  62459, 177777, 238100, 228431,  18795, 158587,  35469,
       147305,   8547,  20458,  23434,  52501,  23157,  14735,  72277,
       142220,   8710, 228134,  64539, 237320,  30007,  16563, 161869,
         8855,  15127, 127500,  16304,  13420,  42967,  64311,   8756])

In [427]:
# remove recipes which are missing in nutrition df from the ingredients tecipes dataframe
mask = recipe_db.index.isin(nutrition_db2.index.values)
recipe_db = recipe_db.loc[mask]


In [428]:
user_recipes_fat = recipe_db[recipe_db.index.isin(subset_fat_id)]
#user_recipes_normal = recipe_db[recipe_db.index.isin(subset_normal_id)]

In [429]:
####temp#######
#temp_fat_20 = [16966,  14753,  62459,  18397,   8757,  52501,   8556, 240522,
      #  86628, 221304, 223596,  51653,  48921, 142220,  76763, 222509,
      #  17496,  14710,  86860,  86813]

#user_recipes_fat = recipe_db[recipe_db.index.isin(temp_fat_20)]

#temp_normal_20 = [36994,  16741,  14622,  24080,  11720,   8994,  24509,   8949,
      # 217899,  15225,   8549, 220214,  21140, 230735, 140135,  27946,
      # 199382, 109782,  30794,  22364]
####temp#######
#user_recipes_normal = recipe_db[recipe_db.index.isin(temp_normal_20)]



In [430]:
#top10 most common ingredients
#16421    2125 salt
#4342     1506 garlic
#4397     1412 onion
#16406    1185 ground black pepper
#16157    1016 butter
#6307      944 olive oil
#6494      786 skinless bone less chicken breast
#2496      749 water
#16238     574 grated Parmesan cheese
#16317     538 eggs


drop_id_list = [16421, 4342, 4397, 16406, 16157, 6307, 6494, 2496, 16238, 16317]





In [431]:
#filter out ingredients which are too common and the recipes in the user profile from the recipes database
new_recipe_db = recipe_db.drop(axis=1, labels=drop_id_list)
new_recipe_db_wo_userrecipes_fat = new_recipe_db.drop(axis=0, labels=subset_fat_id)


In [432]:
#remove basic ingredients like garlic, olive oil, skinless bone less chicken breast, butter, milk, salt and pepper, onion
new_user_recipes_fat = user_recipes_fat.drop(axis=1, labels=drop_id_list)
#new_user_recipes_normal = user_recipes_normal.drop(axis=1, labels=drop_id_list)

In [433]:
sample_fat = new_user_recipes_fat.sample(n=10).copy()


In [434]:
#sample_normal = new_user_recipes_normal.sample(n=20).copy()

In [435]:
## Jaccard - fat

result_array = cdist(sample_fat, new_recipe_db_wo_userrecipes_fat,'jaccard')
result_w_filter_fat = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_fat.index.values, index=sample_fat.index.values)
# result_2

result_w_filter_10_fat = pd.DataFrame(result_w_filter_fat.sum(), columns=['jaccard_distance_sum'])
result_w_filter_10_fat = result_w_filter_10_fat.sort_values(by='jaccard_distance_sum')
result_w_filter_10_fat['jaccard_distance_sum'] = result_w_filter_10_fat['jaccard_distance_sum'].div(10)
result_w_filter_10_fat[0:10]


,jaccard_distance_sum
46813,0.88123
133034,0.88424
16189,0.88773
42919,0.88852
64893,0.88889
238691,0.89352
16998,0.89352
11735,0.89352
240619,0.89462
48477,0.89508


In [436]:
# userrecipes fat
sample_fat.index.to_numpy()

array([106528,  16563, 237320,  17253, 228431, 149738,  72277, 142220,
       228134,  38004])

In [437]:
result_w_filter_10_fat[0:10].index.to_numpy()

array([ 46813, 133034,  16189,  42919,  64893, 238691,  16998,  11735,
       240619,  48477])

In [438]:
result_w_filter_fat_short = result_w_filter_fat.copy()


In [439]:
# top 10 fat short
fat_short = pd.DataFrame(np.sort(result_w_filter_fat_short.values, axis=0), index=result_w_filter_fat_short.index, columns=result_w_filter_fat_short.columns)

new_columns = fat_short.columns[fat_short.loc[fat_short.first_valid_index()].argsort()]
result_fat_short = fat_short[new_columns]
result_fat_short = result_fat_short.reset_index()
result_fat_short = result_fat_short.loc[1].to_frame()

result_fat_short[0:11].index.to_numpy()


array(['index', 13664, 8689, 30679, 218720, 87133, 19427, 231064, 216902,
       17022, 17184], dtype=object)

In [440]:
sample_fat.index.to_numpy()

array([106528,  16563, 237320,  17253, 228431, 149738,  72277, 142220,
       228134,  38004])

In [441]:
#create one dimensional ingredients vector from user recipes profile
user_one_vector_fat = sample_fat.copy()
user_one_vector_fat.loc['sum',:]= user_one_vector_fat.sum(axis=0)
user_one_vector_fat = user_one_vector_fat.drop(axis=0, labels=sample_fat.index)
user_one_vector_fat[user_one_vector_fat > 0] = 1

In [442]:
user_one_vector_fat
user_one_vector_fat.apply(pd.value_counts).count(axis=1)

,111,126,257,443,445,578,615,629,631,858,...,23047,23274,23383,24865,25518,25522,26269,26934,27343,27805
id,,,,,,,,,,,,,,,,,,,,,
sum,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


0.00000    670
1.00000     50
dtype: int64

In [443]:
## Jaccard - Vector fat top 10

result_array = cdist(user_one_vector_fat, new_recipe_db_wo_userrecipes_fat,'jaccard')
result_w_filter_vector_fat = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_fat.index.values, index=user_one_vector_fat.index.values)


result_w_filter_vector_10_fat = pd.DataFrame(result_w_filter_vector_fat.sum(), columns=['jaccard_distance_sum'])
result_w_filter_vector_10_fat = result_w_filter_vector_10_fat.sort_values(by='jaccard_distance_sum')

result_w_filter_vector_10_fat[0:10]


,jaccard_distance_sum
180905,0.83636
53075,0.85455
42919,0.86275
245764,0.86275
70012,0.86538
75377,0.86792
125658,0.86792
22286,0.87037
21670,0.87273
231396,0.87719


In [444]:
####### results

In [445]:
# userrecipes fat
sample_fat.index.to_numpy()

array([106528,  16563, 237320,  17253, 228431, 149738,  72277, 142220,
       228134,  38004])

In [446]:
###### jaccard fat top 10 naiv
result_w_filter_10_fat[0:10].index.to_numpy()

array([ 46813, 133034,  16189,  42919,  64893, 238691,  16998,  11735,
       240619,  48477])

In [447]:
###### jaccard kurz fat top 10
result_fat_short[1:11].index.to_numpy()


array([13664, 8689, 30679, 218720, 87133, 19427, 231064, 216902, 17022,
       17184], dtype=object)

In [448]:
###### jaccard vector fat top 10
result_w_filter_vector_10_fat[0:10].index.to_numpy()

array([180905,  53075,  42919, 245764,  70012,  75377, 125658,  22286,
        21670, 231396])

In [453]:
##### random top 10
subset_random_top10.index.to_numpy()

array([ 70491,  21815,  58165,  17646,  16767,  90451, 229277,  15919,
       148970,  88745])